In [11]:
from openai import OpenAI
from pathlib import Path

key = open('openai.key').read().strip()
client = OpenAI(api_key=key, organization='org-l0QXnTWrsY221IPu8QIF1k1H', project='proj_hdXOnDa32fjKeT330EowFZsc')

In [12]:
pdf_path = Path('public_agenda_june24_.pdf')

In [23]:
message_file = client.files.create(file=pdf_path, purpose='assistants')

In [24]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "list any mentions of technology purchases in the document",
            "attachments": [
                { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
            ],
        }
    ]
)

In [28]:
assistant = client.beta.assistants.create(
    name="Technology Purchases",
    instructions="You are a research assistant. Your task is to list any mentions of technology purchases in the document.",
    model='gpt-4o',
    tools=[{"type": "file_search"}],
)

In [30]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [31]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

The mentions of technology purchases in the document include the following:

1. **Versadex R.M.S. software support and maintenance**:
   - The extended and increased contract with Versaterm for Versadex Records Management System (R.M.S.) software support and maintenance is detailed. The estimated cost for the period from November 5, 2024, to November 4, 2028, is projected to be $4.45 million[0][1][2].

2. **Upgradations to Video Bail Capabilities, Radio & Telecom capabilities**:
   - Introduction and upgradation of Video Bail Capabilities in 5 police divisions.
   - Upgraded Radio & Telecom capabilities are also mentioned as part of the Toronto Police Service Update[3][4].

3. **Public Safety Data Portal and Community Technology**:
   - An investment in modern technology for a public safety data portal to encourage open data for public safety in Toronto.
   - Modern technology tools to improve communication through the Toronto Police Services app, video calling, social media, and vario